In [1]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import config as creds
import psycopg2

In [2]:
#Set up connection to database
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

In [3]:
conn, cursor = connect()

Connected!


In [4]:
# Load the data

file_path = ('diabetes_012_health_indicators_BRFSS2015.csv')
df = pd.read_csv(file_path)
#print(df.shape)

#Convert diabetes to 1 and prediabetes to 0
def diabetes_binary(x, r):
    if x == r:
        return('1')
    if x < r:
        return('0')
    
df['Diabetes_012'] = df["Diabetes_012"].apply(diabetes_binary, args =[2])



df.head(20)

                      

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
5,0,1.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,2.0,0.0,1.0,10.0,6.0,8.0
6,0,1.0,0.0,1.0,30.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,3.0,0.0,14.0,0.0,0.0,9.0,6.0,7.0
7,0,1.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,0.0,0.0,1.0,0.0,11.0,4.0,4.0
8,1,1.0,1.0,1.0,30.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,5.0,30.0,30.0,1.0,0.0,9.0,5.0,1.0
9,0,0.0,0.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,1.0,8.0,4.0,3.0


In [5]:
#Create the lifestyle df
lifestyle_df = df[['Diabetes_012','CholCheck','Smoker','PhysActivity','Fruits','Veggies','HvyAlcoholConsump']]

#Drops NA rows
lifestyle_df.dropna()

,Diabetes_012,CholCheck,Smoker,PhysActivity,Fruits,Veggies,HvyAlcoholConsump
0,0,1.0,1.0,0.0,0.0,1.0,0.0
1,0,0.0,1.0,1.0,0.0,0.0,0.0
2,0,1.0,0.0,0.0,1.0,0.0,0.0
3,0,1.0,0.0,1.0,1.0,1.0,0.0
4,0,1.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...
253675,0,1.0,0.0,0.0,1.0,1.0,0.0
253676,1,1.0,0.0,0.0,0.0,0.0,0.0
253677,0,1.0,0.0,1.0,1.0,0.0,0.0
253678,0,1.0,0.0,0.0,1.0,1.0,0.0


In [6]:
#Split the data into features(x) and Target(y)
y = lifestyle_df["Diabetes_012"]
X = lifestyle_df.drop(['Diabetes_012'], axis = 1)

In [7]:
lifestyle_df['Diabetes_012'] = lifestyle_df['Diabetes_012'] .astype('int64')

C:\Users\kylen\miniconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
#Check data types
lifestyle_df.dtypes

Diabetes_012           int64
CholCheck            float64
Smoker               float64
PhysActivity         float64
Fruits               float64
Veggies              float64
HvyAlcoholConsump    float64
dtype: object

In [9]:
X.describe()

,CholCheck,Smoker,PhysActivity,Fruits,Veggies,HvyAlcoholConsump
count,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000,253680.000000
mean,0.962670,0.443169,0.756544,0.634256,0.811420,0.056197
std,0.189571,0.496761,0.429169,0.481639,0.391175,0.230302
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
50%,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
y.value_counts()

0    218334
1     35346
Name: Diabetes_012, dtype: int64

In [11]:
#Split the data into test and training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50,random_state=1, stratify=y)
Counter(y_train)

Counter({'0': 109167, '1': 17673})

In [12]:
X_test

,CholCheck,Smoker,PhysActivity,Fruits,Veggies,HvyAlcoholConsump
5404,1.0,1.0,1.0,0.0,1.0,1.0
134844,1.0,0.0,1.0,1.0,1.0,0.0
129255,1.0,0.0,1.0,1.0,1.0,0.0
224524,1.0,0.0,1.0,1.0,1.0,0.0
92752,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...
103802,1.0,1.0,0.0,0.0,1.0,0.0
7476,1.0,1.0,1.0,1.0,1.0,0.0
96653,1.0,0.0,1.0,1.0,1.0,0.0
87326,1.0,1.0,1.0,1.0,1.0,0.0


In [13]:
#Initialize Logistic Regression Model
model = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)



In [14]:
#Fit the model
model.fit(X_train,y_train)

LogisticRegression(max_iter=200, random_state=1)

In [15]:
predictions = model.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test})
results.head(10)



,Prediction,Actual
5404,0,0
134844,0,0
129255,0,0
224524,0,0
92752,0,0
151708,0,0
55470,0,1
30688,0,0
94938,0,0
245034,0,0


In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.8606669820245979

In [17]:
from imblearn.metrics import classification_report_imbalanced
matrix = confusion_matrix(y_test, predictions)
print(matrix)

[[109167      0]
 [ 17673      0]]


In [18]:
report = classification_report(y_test, predictions)
print(report)

C:\Users\kylen\miniconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kylen\miniconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.86      1.00      0.93    109167
           1       0.00      0.00      0.00     17673

    accuracy                           0.86    126840
   macro avg       0.43      0.50      0.46    126840
weighted avg       0.74      0.86      0.80    126840



C:\Users\kylen\miniconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


   ### Oversampling

In [19]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)



In [20]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)



LogisticRegression(random_state=1)

In [21]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5927022262663242

In [22]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[71176, 37991],
       [ 8246,  9427]], dtype=int64)

In [23]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.90      0.65      0.53      0.75      0.59      0.35    109167
          1       0.20      0.53      0.65      0.29      0.59      0.34     17673

avg / total       0.80      0.64      0.55      0.69      0.59      0.35    126840



In [24]:
create_table = """
    CREATE TABLE IF NOT EXISTS lifestyle_df(

Diabetes_012         INTEGER
CholCheck            FLOAT,
Smoker                FLOAT,
PhysActivity         FLOAT,
Fruits                FLOAT,
Veggies               FLOAT,
HvyAlcoholConsump     FLOAT
    )
    """

In [25]:
# Execute SQL Command and commit to DB
conn.commit()